# Protein Folding | EPOCH Lab 2022

In [1]:
%matplotlib inline

from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

import nglview as nv

In [2]:
try:
  platform = Platform.getPlatformByName("CUDA")
except Exception:
  platform = Platform.getPlatformByName("OpenCL")

In [3]:
pdb = PDBFile('pdb/input.pdb')
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')
# forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

In [4]:
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens(forcefield)
modeller.topology

<Topology; 1 chains, 2798 residues, 8867 atoms, 6111 bonds>

In [5]:
unmatched_residues = forcefield.getUnmatchedResidues(pdb.topology)
unmatched_residues

[]

In [6]:
system = forcefield.createSystem(modeller.topology,
                                 nonbondedMethod=PME,
                                 nonbondedCutoff=1*nanometer,
                                 constraints=HBonds)

In [7]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)

In [8]:
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('tmp/output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True))

In [9]:
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-124175.47391541512,250.7643628022183
2000,-117939.25516541512,283.9235004604744
3000,-115919.97391541512,292.1637977108803
4000,-115514.16141541512,296.9147166863754
5000,-115331.16141541512,295.04614019426384
6000,-115326.34891541512,301.097832138808
7000,-115159.78641541512,299.456413457527
8000,-115589.69266541512,301.59132655969705
9000,-115411.44266541512,298.01842409657604
10000,-115807.59891541512,295.1537031720123


In [10]:
view = nv.show_file('tmp/output.pdb')
view

NGLWidget()